# Biomedical LLM Evaluation Suite

**Project:** Pipeline Optimisation
**Purpose:** Evaluate language models on literature interpretation tasks

In [ ]:
# Login to HuggingFace (only needed for gated models like Meditron, Llama)
# Get your token from: https://huggingface.co/settings/tokens

from huggingface_hub import login

# Option 1: Interactive login (recommended for Colab)
login()

# Option 2: Login with token directly (uncomment and add your token)
# login(token='hf_YOUR_TOKEN_HERE')

print('✓ Logged in to HuggingFace')

In [ ]:
import json
import time
from pathlib import Path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import torch
from tqdm.notebook import tqdm

print('✓ Imports successful')
print(f"Device: {'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Load test datasets
test_dir = Path('model_tests')

with open(test_dir / 'test_relevance.json') as f:
    relevance_data = json.load(f)

with open(test_dir / 'test_mechanism.json') as f:
    mechanism_data = json.load(f)

with open(test_dir / 'test_quality.json') as f:
    quality_data = json.load(f)

with open(test_dir / 'test_stability.json') as f:
    stability_data = json.load(f)

print(f'✓ Loaded test data:')
print(f'  - Relevance: {len(relevance_data)} items')
print(f'  - Mechanism: {len(mechanism_data)} items')
print(f'  - Quality: {len(quality_data)} items')
print(f'  - Stability: {len(stability_data)} items')

In [ ]:
# Preview a test item
sample = relevance_data[0]
print(f"ID: {sample['id']}")
print(f"Agent: {sample['agent']}")
print(f"Pathway: {sample['pathway']}")
print(f"Gold Label: {sample['gold_label']}")
print(f"\nAbstract (first 200 chars):\n{sample['abstract'][:200]}...")

In [ ]:
class ModelEvaluator:
    def __init__(self, model_name, device='auto'):
        self.model_name = model_name
        self.device = device if device != 'auto' else ('cuda' if torch.cuda.is_available() else 'cpu')
        
        print(f'Loading model: {model_name}')
        print(f'Device: {self.device}')
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=True,
                torch_dtype=torch.float16 if self.device == 'cuda' else torch.float32,
                device_map=self.device if self.device == 'cuda' else None
            )
            self.is_causal = True
        except:
            self.model = AutoModelForSeq2SeqLM.from_pretrained(
                model_name,
                trust_remote_code=True,
                torch_dtype=torch.float16 if self.device == 'cuda' else torch.float32,
                device_map=self.device if self.device == 'cuda' else None
            )
            self.is_causal = False
        
        if self.device == 'cpu':
            self.model = self.model.to(self.device)
        
        self.model.eval()
        
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        if hasattr(self.model.config, 'max_position_embeddings'):
            self.model_max_length = self.model.config.max_position_embeddings
        elif hasattr(self.model.config, 'n_positions'):
            self.model_max_length = self.model.config.n_positions
        else:
            self.model_max_length = 1024
        
        print(f'✓ Model loaded (max length: {self.model_max_length})')
    
    def generate_response(self, prompt, max_new_tokens=256):
        safe_input_length = self.model_max_length - max_new_tokens - 10
        
        inputs = self.tokenizer(prompt, return_tensors='pt', truncation=True, max_length=safe_input_length)
        inputs = {k: v.to(self.device) for k, v in inputs.items() if k != 'token_type_ids'}
        
        try:
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    temperature=0.1,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
            
            full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            if self.is_causal:
                response = full_text[len(prompt):].strip()
            else:
                response = full_text.strip()
            
            return response
        except Exception as e:
            return f'{{"error": "Generation failed: {str(e)}"}}'
    
    def extract_json(self, response):
        try:
            if '```json' in response:
                start = response.find('```json') + 7
                end = response.find('```', start)
                json_str = response[start:end].strip()
            elif '{' in response:
                start = response.find('{')
                end = response.rfind('}') + 1
                json_str = response[start:end]
            else:
                json_str = response
            
            return json.loads(json_str)
        except Exception as e:
            return {'error': f'JSON parsing failed: {str(e)}', 'raw_response': response}

print('✓ ModelEvaluator class defined')

## Load Model

**Recommended models:**
- `facebook/galactica-1.3b` - Best for Colab free tier
- `facebook/galactica-125m` - Fastest, may struggle
- `BioMistral/BioMistral-7B` - Best accuracy, needs Colab Pro

In [ ]:
# Change model here
MODEL_NAME = 'facebook/galactica-1.3b'

evaluator = ModelEvaluator(MODEL_NAME)

## Quick Test (2 items)

Test with 2 items first to verify the model works

In [ ]:
prompt_template = '''You are a biomedical expert analyzing scientific literature.

Task: Determine if the following PubMed abstract provides mechanistic evidence relating the therapeutic agent to the specified pathway.

Agent: {agent}
Pathway: {pathway}

Abstract: {abstract}

Provide your assessment as a JSON object with the following structure:
{{
  "relevance": "relevant" or "not_relevant",
  "rationale": "brief explanation"
}}

JSON Response:'''

quick_test = relevance_data[:2]
results = []

for item in tqdm(quick_test, desc='Quick Test'):
    prompt = prompt_template.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    start = time.time()
    response = evaluator.generate_response(prompt)
    elapsed = time.time() - start
    
    parsed = evaluator.extract_json(response)
    predicted = parsed.get('relevance', '').lower()
    expected = item['gold_label'].lower()
    
    results.append({
        'id': item['id'],
        'expected': expected,
        'predicted': predicted,
        'correct': predicted == expected,
        'time': elapsed
    })
    
    print(f"{item['id']}: {predicted} (expected: {expected}) - {'✓' if predicted == expected else '✗'} [{elapsed:.1f}s]")
    print(f"Response: {response[:150]}...\n")

accuracy = sum(r['correct'] for r in results) / len(results)
print(f"\nQuick Test Accuracy: {accuracy:.1%}")
print(f"Avg Time: {sum(r['time'] for r in results)/len(results):.1f}s per item")

## Test 1: Relevance Assessment (10 items)

In [ ]:
relevance_results = []

for item in tqdm(relevance_data, desc='Relevance Test'):
    prompt = prompt_template.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    start = time.time()
    response = evaluator.generate_response(prompt)
    elapsed = time.time() - start
    
    parsed = evaluator.extract_json(response)
    predicted = parsed.get('relevance', '').lower()
    expected = item['gold_label'].lower()
    
    relevance_results.append({
        'id': item['id'],
        'agent': item['agent'],
        'pathway': item['pathway'],
        'expected': expected,
        'predicted': predicted,
        'correct': predicted == expected,
        'time': elapsed
    })

df_relevance = pd.DataFrame(relevance_results)
display(df_relevance)

relevance_accuracy = df_relevance['correct'].mean()
print(f"\nRelevance Accuracy: {relevance_accuracy:.1%}")
print(f"Correct: {df_relevance['correct'].sum()}/{len(df_relevance)}")
print(f"Avg Time: {df_relevance['time'].mean():.1f}s per item")

## Test 2: Evidence Quality (5 items)

In [ ]:
quality_prompt = '''You are a biomedical expert evaluating scientific evidence.

Task: Assess the strength of evidence in this abstract.

Evidence Quality Criteria:
- Strong: Multiple experimental approaches, rigorous controls, clinical validation
- Moderate: In vitro data with multiple assays
- Weak: Limited experimental data
- Insufficient: Minimal data, unclear methods

Agent: {agent}
Pathway: {pathway}

Abstract: {abstract}

Provide assessment as JSON:
{{
  "evidence_quality": "strong" or "moderate" or "weak" or "insufficient",
  "justification": "brief explanation"
}}

JSON Response:'''

quality_results = []

for item in tqdm(quality_data, desc='Quality Test'):
    prompt = quality_prompt.format(
        agent=item['agent'],
        pathway=item['pathway'],
        abstract=item['abstract']
    )
    
    start = time.time()
    response = evaluator.generate_response(prompt)
    elapsed = time.time() - start
    
    parsed = evaluator.extract_json(response)
    predicted = parsed.get('evidence_quality', '').lower()
    expected = item['gold_label'].lower()
    
    quality_results.append({
        'id': item['id'],
        'expected': expected,
        'predicted': predicted,
        'correct': predicted == expected,
        'time': elapsed
    })

df_quality = pd.DataFrame(quality_results)
display(df_quality)

quality_accuracy = df_quality['correct'].mean()
print(f"\nQuality Accuracy: {quality_accuracy:.1%}")
print(f"Correct: {df_quality['correct'].sum()}/{len(df_quality)}")

## Summary and Visualization

In [ ]:
# Summary
print(f"{'='*80}")
print(f"EVALUATION SUMMARY: {MODEL_NAME}")
print(f"{'='*80}")
print(f"Relevance Accuracy:        {relevance_accuracy:.1%}")
print(f"Evidence Quality Accuracy: {quality_accuracy:.1%}")
print(f"Avg Inference Time:        {df_relevance['time'].mean():.1f}s per item")
print(f"{'='*80}")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy comparison
accuracies = {'Relevance': relevance_accuracy, 'Quality': quality_accuracy}
axes[0].bar(accuracies.keys(), accuracies.values(), color=['#1f77b4', '#ff7f0e'])
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Performance by Test Type')
axes[0].set_ylim([0, 1])
axes[0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5)

# Time distribution
all_times = list(df_relevance['time']) + list(df_quality['time'])
axes[1].hist(all_times, bins=15, color='#1f77b4', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Time (seconds)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Inference Time Distribution')

plt.tight_layout()
plt.show()

# Save results
summary_df = pd.DataFrame([{
    'Model': MODEL_NAME,
    'Relevance_Accuracy': f"{relevance_accuracy:.1%}",
    'Quality_Accuracy': f"{quality_accuracy:.1%}",
    'Avg_Time_s': f"{df_relevance['time'].mean():.1f}"
}])

display(summary_df)

# Download results (works on Colab)
try:
    from google.colab import files
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"{MODEL_NAME.replace('/', '_')}_{timestamp}_results.csv"
    summary_df.to_csv(filename, index=False)
    files.download(filename)
    print(f'\n✓ Downloaded: {filename}')
except:
    print('\n(Not on Colab - results not auto-downloaded)')